# Step 1. Task 설정하기

## 라이브러리 불러오기

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout,\
GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

# Step 2. 데이터셋 준비하기


## Step 2-1 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls 

drive  gdrive  sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls '/content/gdrive/MyDrive/catdog'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
dataset  desktop.ini  Practice_CatDog.zip


## Step 2-2. 데이터 전처리 하기.
### 훈련데이터 전처리 하기: 스케일링 및 데이터 증강
 - 이미지 데이터를 255로 나누어 0~1 사이의 값으로 맞추어 준다
 - 데이터를 증강한다

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/catdog/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary') #_directory:라벨이 나눠져서 폴더별로있는경우, 바로 구분해서 끌고옴 // targetsize:알아서downsizing해줌 // 여기서는 개or고양이이므로 binary Q.아니라면?파라메터조정을 어떻게해야되는지 알아보기

Found 6803 images belonging to 2 classes.


### 테스트데이터 전처리 하기

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255) #nomalize
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/catdog/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1184 images belonging to 2 classes.


# Step 3. 모델 훈련하기



## Step 3-1. 모델 설계하기

### Stage 1. 특성 추출

In [ ]:
cnn = tf.keras.models.Sequential()

# 컨볼루션 1층
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))


# 컨볼루션-풀링 2층
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))


Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Stage 2. - Full Connection(ANN)

In [ ]:
#ANN의 입력층
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
#ANN의 출력층
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Step 3-2 모델 컴파일

In [ ]:
cnn.compile(optimizer = 'adam', 
            loss = 'binary_crossentropy', 
            metrics = ['accuracy'])

## Step 3-3 모델 훈련

In [ ]:
cnn.fit(x = training_set,
        validation_data = test_set,
        epochs = 25)

Epoch 1/25
213/213 [==============================] - 939s 4s/step - loss: 0.7225 - accuracy: 0.6241 - val_loss: 0.7820 - val_accuracy: 0.4721
Epoch 2/25
213/213 [==============================] - 38s 178ms/step - loss: 0.6017 - accuracy: 0.6919 - val_loss: 0.6309 - val_accuracy: 0.6208
Epoch 3/25
213/213 [==============================] - 37s 175ms/step - loss: 0.5700 - accuracy: 0.7106 - val_loss: 0.9608 - val_accuracy: 0.6039
Epoch 4/25
213/213 [==============================] - 37s 174ms/step - loss: 0.5369 - accuracy: 0.7344 - val_loss: 0.5371 - val_accuracy: 0.7500
Epoch 5/25
213/213 [==============================] - 38s 176ms/step - loss: 0.5050 - accuracy: 0.7575 - val_loss: 0.5349 - val_accuracy: 0.7373
Epoch 6/25
213/213 [==============================] - 37s 175ms/step - loss: 0.4785 - accuracy: 0.7729 - val_loss: 0.4098 - val_accuracy: 0.8209
Epoch 7/25
213/213 [==============================] - 40s 188ms/step - loss: 0.4632 - accuracy: 0.7820 - val_loss: 0.5736 - val_accu

## Step 5. 모델 활용하기

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/gdrive/MyDrive/catdog/dataset/single_prediction/cat_dog_3.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(result)
print(prediction)

AttributeError: ignored

 ### 학습한 모델 저장하기

In [ ]:
# 기본폴더에 모델을 저장하면 데이터가 사라진다.
# 원하는 경로를 입력한 뒤 모델을 저장해 준다.

# 작업폴더 변경
%cd '/content/gdrive/MyDrive'
# 훈련한 cnn 모델을 "cat_dog.h5" 이라는 이름으로 저장해 준다.
cnn.save("cat_dog.h5")
# 저장이 제대로 되었는지 확인한다.
!ls

/content/gdrive/MyDrive
 ㄱㅇ
'18차시_CNN_CIFAR_Improved.ipynb의 사본'
'제목 없는 문서 (1).gdoc'
 1.gdoc
 1.jpg
'20차시_LSTM_imdb_lstm.ipynb의 사본'
'21차시_LSTM_Stock_Returns.ipynb의 사본'
 2.gdoc
'인공지능활용분석2급_실기_시험_소스코드_수강생제공용.ipynb의 사본'
 2.jpg
'2._인공지능 능력자 with Python_코딩기반_전처리_시각화_인공지능합본.pdf'
 3.jpg
 4.jpg
 5.jpg
 6.jpg
 7.jpg
 CASE_NIKE.pdf
 catdog
'cat dog'
 cat_dog.h5
'Colab Notebooks'
 DevKurly
'제목 없는 문서.gdoc'
'제목 없는 문서의 번역된 사본.gdoc'
 이력서.gdoc
'제목 없는 스프레드시트.gsheet'
'Numpy Stack.'
'ONLYFRESH (1).gslides'
'OnlyFresh-2 (1).pdf'
 OnlyFresh-2.pdf
 ONLYFRESH.gslides
'Team DevKurly 화면정의서 (1).gdoc'
'Team DevKurly 화면정의서.gdoc'
 Untitled
 Untitled0.ipynb
 walmart-china.pdf


## 저장해둔 모델로 분석하기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd '/content/gdrive/MyDrive'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive


In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model

# 모델이 저장되어있는 폴더의 경로르 입력해 준다.
%cd '/content/gdrive/MyDrive'
#저장된 모델을 불러온다
cnn = load_model('cat_dog.h5')

#분석할 이미지의 경로를 입력하고 불러온다.
test_image = image.load_img('/content/gdrive/MyDrive/catdog/dataset/single_prediction/cat_or_dog_1.jpg',
                            target_size = (64, 64))
#불러온 이미지를 어레이 데이터로 바꾼다
test_image = image.img_to_array(test_image)
#테스트 이미지의 차원을 확장한다.
test_image = np.expand_dims(test_image, axis = 0)
# 학습한 모델로 예측한 결과물을 result 변수에 저장해 준다.
result = cnn.predict(test_image)

# training_set.class_indices
if result[0][0] == 0:
    prediction = 'cat'
else:
    prediction = 'dog'

print(result)
print(prediction)

/content/gdrive/MyDrive


AttributeError: ignored